In [1]:
import random
import csv

from psycopg2 import connect, extras

# from libs.connectors.postgres_connector import PostgresConnector
# from libs.feeds.postgres_feed import PostgresFeed

from manglers.mangle_org_name import mangle_org_name
from manglers.mangle_url import mangle_url
from manglers.mangle_address import mangle_address
from manglers.mangle_region import mangle_region
from manglers.mangle_country import mangle_country
import manglers.mangler_parameters

# connection details for source data table
source_host = 'localhost'
source_user = 'postgres'
source_passwd = 'postgres'
source_db = 'defaultdb'
source_port = 5432

# connection details for training data table
training_host = 'localhost'
training_user = 'postgres'
training_passwd = 'postgres'
training_db = 'silobuster_testing'
training_port = 5432
training_table = 'organizations_mangled_110422'
training_set_num = 1

replace_train_data = 'Y'

csv_output = 'training_sets/training_set_110422.csv'

print ('Starting connections...')
source_conn = connect(
        database=source_db,
        user=source_user,
        password=source_passwd,
        host=source_host,
        port=source_port
    )

training_conn = connect(
        database=training_db,
        user=training_user,
        password=training_passwd,
        host=training_host,
        port=training_port
    )

# defining SQL queries to run to create/replace training table, and inserting original and mangled rows
create_qry = f"""
    create table if not exists {training_table} (
        id Serial,
        name VARCHAR,
        description VARCHAR,
        url VARCHAR,
        address_1 VARCHAR,
        address_2 VARCHAR,
        city VARCHAR,
        state_province VARCHAR,
        postal_code VARCHAR,
        country VARCHAR,
        type VARCHAR,
        region VARCHAR,
        duplicate_id VARCHAR,
        duplicate_type VARCHAR,
        training_set VARCHAR
    )
"""
drop_qry = f"DROP TABLE IF EXISTS {training_table}"
select_qry = "select t1.name, t1.description, t1.url, t3.address_1, t3.address_2, t3.city, t3.region, t3.state_province, t3.postal_code, t3.country, t3.type from organization t1 left join location t2 on t1.id = t2.organization_id left join address t3 on t3.location_id = t2.id"
insert_qry = f"INSERT INTO {training_table} (name, description, url, address_1, address_2, city, state_province, postal_code, country, type, region, training_set) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING id"
insert_dup_qry = f"INSERT INTO {training_table} (name, description, url, address_1, address_2, city, state_province, postal_code, country, type, region, duplicate_id, duplicate_type, training_set) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

with training_conn.cursor() as training_cur:
    if replace_train_data == 'Y':
        training_cur.execute(drop_qry)
    training_cur.execute(create_qry)
    training_conn.commit()

print ("Connected?")

### Data keys for mangling

# bringing in comprehensive list of all possible address suffixes
# each row of suffixes contains list of possible versions of a specific suffix
# e.g. one row has Street,ST,st,st.,Str
address_suffixes = []
with open('./helper_data/street_suffixes.csv', 'r') as suffixes_file:
    csv_reader = csv.reader(suffixes_file)
    for row in csv_reader:
        row_clean = [suffix for suffix in row if suffix] # removing empty strings
        address_suffixes.append(row_clean)

state_keys = list()
state_keys.append(['wa', 'wash', 'washington'])

with source_conn.cursor(cursor_factory=extras.RealDictCursor) as source_cur:
    source_cur.execute(select_qry)
    data = source_cur.fetchall()
    print ("Retrieved data...")
    
    dup_address = random.randint(1,3)
    dup_blank_stuff = random.randint(1,2)
    
    for count, row in enumerate(data):
        
        print (f"Insert row count: {count}")
        
        # removing trailing/leading spaces and converting to lowercase
        name = row['name'].strip().lower()
        try:
            desc = row['description'].strip().lower()
        except:
            desc = ''
        try:
            url = row['url'].strip().lower()
        except:
            url = ''
        try:
            address_1 = row['address_1'].strip().lower()
        except:
            address_1 = ''
        try:
            address_2 = row['address_2'].strip().lower()
        except:
            address_2 = ''
        try:
            city = row['city'].strip().lower()
        except:
            city = ''
        try:
            region = row['region'].strip().lower()
        except:
            region = ''
        try:
            state = row['state_province'].strip().lower()
        except:
            state = ''
        try:
            postal = row['postal_code'].strip().lower()
        except:
            postal = ''
        try:
            country = row['country'].strip().lower()
        except:
            country = ''
        try:
            type_row = row['type'].strip().lower()
        except:
            type_row = ''
        
        # Write the rows to the mangled table
        with training_conn.cursor() as training_cur:
            training_cur.execute(insert_qry, [
                    name, 
                    desc, 
                    url, 
                    address_1, 
                    address_2, 
                    city, 
                    state, 
                    postal,
                    country, 
                    type_row,
                    region,
                    training_set_num

            ])
            insert_id = training_cur.fetchone()[0]
            training_conn.commit()
        
        
        with open(csv_output, 'a') as csvfile:
            w = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
            w.writerow([
                name,
                desc,
                url,
                address_1,
                address_2,
                city,
                state,
                postal,
                country,
                type_row,
                region,
                '',
                '',
                training_set_num,
            ])

        # mangling organization name, region, country, URL, address_1
        mangled_name = mangle_org_name(name,  
            remove_prob = manglers.mangler_parameters.name_remove_char_prob, 
            replace_prob = manglers.mangler_parameters.name_replace_char_prob, 
            null_prob = manglers.mangler_parameters.name_nullify_prob
        )
        mangled_region = mangle_region(region, 
            remove_prob = manglers.mangler_parameters.region_remove_char_prob, 
            replace_prob = manglers.mangler_parameters.region_replace_char_prob, 
            null_prob = manglers.mangler_parameters.region_nullify_prob
        )
        mangled_country = mangle_country(country, 
            remove_prob = manglers.mangler_parameters.country_remove_char_prob, 
            replace_prob = manglers.mangler_parameters.country_replace_char_prob, 
            null_prob = manglers.mangler_parameters.country_nullify_prob
        )
        mangled_url = mangle_url(url, 
            probs_dict = manglers.mangler_parameters.url_mangling_probs_dict, 
            tld_swap_prob_dict = manglers.mangler_parameters.tld_swap_prob_dict
        )
        mangled_address_1 = mangle_address(address_1, 
            suffixes = address_suffixes, 
            swap_suffix_prob = manglers.mangler_parameters.address_suffix_swap_prob, 
            blank_prob = manglers.mangler_parameters.address_nullify_prob
        )

        # assign tags based on how field(s) were mangled in duplicated row
        duplicate_type = []
        original_fields = [name, address_1, url, region, country]
        mangled_fields = [mangled_name, mangled_address_1, mangled_url, mangled_region, mangled_country]
        duplicate_tags = ['mangled_name', 'mangled_address_1', 'mangled_url', 'mangled_region', 'mangled_country']

        for orig_field, mangled_field, tag in zip(original_fields, mangled_fields, duplicate_tags):
            if orig_field != mangled_field:
                duplicate_type.append(tag)
        
        # if any of the fields were mangled, create a duplicate row in database and in training set CSV file
        if len(duplicate_type) > 0:  
            dup_row = {
                'name': mangled_name,
                'description': desc,
                'url': mangled_url,
                'address_1': mangled_address_1,
                'address_2': address_2,
                'city': city,
                'region': mangled_region,
                'state': state,
                'postal': postal,
                'country': mangled_country,
                'duplicate_id': insert_id,
                'duplicate_type': duplicate_type,
                'training_set': training_set_num
            }

            # insert duplicated row into training/mangled table
            with training_conn.cursor() as dup1_cur:
                dup1_cur.execute(insert_dup_qry, [
                        mangled_name,
                        desc,
                        mangled_url,
                        mangled_address_1,
                        address_2,
                        city,
                        state,
                        postal,
                        mangled_country,
                        type_row,
                        mangled_region,
                        insert_id,
                        '|'.join(duplicate_type),
                        training_set_num,
                    ])
                
    
            # write duplicated row to CSV file
            with open(csv_output, 'a') as csvfile:
                w = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
                w.writerow([
                    mangled_name,
                    desc,
                    mangled_url,
                    mangled_address_1,
                    address_2,
                    city,
                    state,
                    postal,
                    mangled_country,
                    type_row,
                    mangled_region,
                    insert_id,
                    '|'.join(duplicate_type),
                    training_set_num,
                ])
                
 
print ('finished')           
            

ModuleNotFoundError: No module named 'tld'